# Dream Destination Finder with CrewAI and Amazon Bedrock

In this notebook, we will explore how to use the CrewAI framework with Amazon Bedrock to build an intelligent agent that can find dream travel destinations based on user preferences. The agent will utilize a large language model (LLM) and web search capabilities to research and recommend destinations that match the user's description.

### Prerequisites

Before we begin, make sure you have the following installed:
`boto3` and `botocore` for interacting with AWS services
`crewai` and `crewai_tools` for building agentic workflows

In [ ]:
# !pip install boto3==1.34.162 botocore==1.34.162 crewai==0.70.1 crewai_tools==0.14.0 duckduckgo-search==6.3.1 unstructured==0.16.6 PyPDF2==3.0.1

__!pip install boto3 botocore crewai crewai_tools duckduckgo-search langchain-community -q__

We start by importing the necessary modules from the crewai and crewai_tools packages.

#### Configuring AWS Credentials:
Before using Amazon Bedrock, ensure that your AWS credentials are configured correctly. You can set them up using the AWS CLI or by setting environment variables. For this notebook, we’ll assume that the credentials are already configured.

To use bedrock we will use [__CrewAI__ __LLM__ api](https://docs.crewai.com/how-to/llm-connections#supported-providers) 

In [40]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Define web-search tool

In [41]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Configuring the LLM

We will use Anthropic’s Claude-3 model via Amazon Bedrock as our LLM. CrewAI uses LiteLLM under the hood to interact with different LLM providers.


In [42]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0")

### Defining the Agent

We will create an agent with the role of a “Travel Destination Researcher.” This agent will be responsible for finding destinations that match the user’s travel preferences.

In [43]:
# Define the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

### Defining the Task

We need to specify the task that the agent will perform. The task includes a description, expected output, and is assigned to the agent we just created.

In [44]:
# Define the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions.",
    agent=travel_agent
)

### Creating the Crew

A crew is a team of agents working together to achieve a common goal. In this case, we have only one agent, but the framework allows for scalability.


In [45]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)

### Executing the Workflow

Now, we can execute the crew with the user’s travel preferences as input.

In [46]:
# User input for travel preferences
user_input = {
    "preferences": "I want a tropical beach vacation with great snorkeling and vibrant nightlife."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find suitable travel destinations based on the user's preferences for a tropical beach vacation with great snorkeling and vibrant nightlife, I will need to search for information on destinations that meet those criteria.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with great snorkeling and nightlife\"}"
## Tool Output: 
While Los Cabos is known for choppy waters that aren't necessarily inviting to swimmers, Todos Santos has several great beaches for swimming and snorkeling (try Playa Los Cerritos and Punta Lobos ... Browse the best tropical vacations around the world, from remote islands to urban hot spots. ... nightlife, and world-class beaches combine

#### As the crew executes, CrewAI will:

•	Decompose the task into actions using ReAct (Reasoning and Act), optionally using the tools assigned to the agent.

•	Make multiple calls to Amazon Bedrock to complete each step from the previous phase.

In [47]:
from IPython.display import Markdown

In [48]:
Markdown(result.raw)

1. Puerto Rico - With its vibrant nightlife, world-class beaches, and excellent snorkeling opportunities, Puerto Rico is an ideal destination for a tropical beach vacation. The island offers a unique blend of Caribbean culture, rich history, and diverse natural landscapes, making it a perfect choice for those seeking a well-rounded tropical experience.

2. St. Croix, U.S. Virgin Islands - St. Croix is renowned for its stunning beaches and exceptional snorkeling spots, such as the Frederiksted Pier, where visitors can explore vibrant coral reefs and encounter a wide variety of marine life. The island also boasts a lively nightlife scene, with plenty of bars, restaurants, and entertainment options to enjoy after a day of beachside relaxation.

3. Aruba - Located in the southern Caribbean Sea, Aruba is a dream destination for snorkelers, with numerous spots suitable for all skill levels. The island's dry climate ensures year-round snorkeling opportunities, and its beaches are among the most beautiful in the region. Aruba also offers a vibrant nightlife scene, with a range of bars, clubs, and restaurants catering to diverse tastes.

4. Cancun, Mexico - Cancun is a popular tropical destination known for its white sandy beaches, crystal-clear waters, and excellent snorkeling opportunities. The nearby Isla Mujeres and the Underwater Museum of Art (MUSA) offer unique snorkeling experiences. Additionally, Cancun is famous for its lively nightlife, with numerous bars, clubs, and entertainment venues along the Hotel Zone.

5. Bali, Indonesia - Bali is a tropical paradise renowned for its stunning beaches, lush landscapes, and vibrant culture. The island offers world-class snorkeling opportunities, with diverse marine life and coral reefs to explore. Bali is also known for its lively nightlife, with numerous beach clubs, bars, and restaurants catering to a variety of tastes and budgets.

These destinations offer a perfect blend of tropical beauty, exceptional snorkeling experiences, and vibrant nightlife, making them ideal choices for those seeking an unforgettable beach vacation filled with adventure, relaxation, and entertainment.

### Adding Memory to the Agent
CrewAI supports [several memory types](https://docs.crewai.com/concepts/memory#implementing-memory-in-your-crew), which help agents remember and learn from past interactions. In this case, we’ll enable short-term memory using Amazon Bedrock’s embedding model.

In [49]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

2024-11-22 16:44:11,111 - 8615497536 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [50]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find suitable travel destinations matching the user's preferences for a tropical beach vacation with great snorkeling and vibrant nightlife, I will search for relevant information on destinations known for those attributes.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"best tropical beach destinations for snorkeling and nightlife\"}"
## Tool Output: 
9 Active Beach Vacations With Snorkeling, Hiking, and Endless Water Sports ... the globe for the best active beach vacations, from over-water rock climbing in Thailand to hiking in Acadia National ... Browse the best tropical vacations around the world, from remote islands to urban hot spots. ... nightlife, and world-class beaches combi

In [51]:
Markdown(result_with_memory.raw)

1. Puerto Rico:
Puerto Rico offers a vibrant blend of tropical beaches, excellent snorkeling opportunities, and a lively nightlife scene. The island's west coast, particularly around Rincón and Isabela, features beautiful sandy beaches and coral reefs teeming with marine life, making it a snorkeler's paradise. Meanwhile, the capital San Juan is famous for its lively nightlife, with bars, clubs, and cultural events catering to diverse tastes.

2. Cancun, Mexico:
Cancun is a renowned tropical destination famous for its pristine beaches, crystal-clear waters, and vibrant nightlife. The nearby Isla Mujeres and the Underwater Museum of Art provide exceptional snorkeling experiences, with an abundance of colorful fish and coral formations. After a day of snorkeling, visitors can indulge in Cancun's energetic party scene, with numerous bars, clubs, and live music venues along the hotel zone.

3. Koh Tao, Thailand:
Koh Tao, a small island in the Gulf of Thailand, is a snorkeler's paradise. Its vibrant coral reefs, such as Tanote Bay and Mango Bay, are home to a diverse array of marine life, including colorful fish, sea turtles, and sharks. Despite its laid-back vibe during the day, Koh Tao transforms at night, offering a lively nightlife scene with beach parties, fire shows, and bustling bars.

4. Bali, Indonesia:
Bali is a tropical paradise that combines stunning beaches, world-class snorkeling, and a vibrant cultural scene. The island's eastern coast, particularly around Amed and Tulamben, is renowned for its excellent snorkeling spots, where visitors can explore shipwrecks and vibrant coral gardens. Bali's nightlife is centered in areas like Seminyak and Kuta, with beach clubs, bars, and lively nightclubs catering to various tastes.

5. St. Croix, U.S. Virgin Islands:
St. Croix, one of the U.S. Virgin Islands, offers a perfect mix of tropical beauty, snorkeling adventures, and nightlife entertainment. The island's western coast, particularly around Frederiksted Pier and Buck Island, provides exceptional snorkeling opportunities, with vibrant coral reefs and abundant marine life. After a day in the water, visitors can explore the lively nightlife in Christiansted, with its bars, restaurants, and live music venues.

### Integrating Retrieval-Augmented Generation (RAG) with Amazon Bedrock Knowledge Base
In this section, we will enhance our dream destination finder agent by incorporating Retrieval-Augmented Generation (RAG) using Amazon Bedrock’s Knowledge Base. This will allow our agent to access up-to-date and domain-specific travel information, improving the accuracy and relevance of its recommendations.



#### What is Retrieval-Augmented Generation (RAG)?

RAG is a technique that combines the capabilities of large language models (LLMs) with a retrieval mechanism to fetch relevant information from external data sources. By integrating RAG, our agent can retrieve the most recent and specific information from a knowledge base, overcoming the limitations of LLMs that may have outdated or insufficient data.

Setting Up Amazon Bedrock Knowledge Base

Before we proceed, ensure you have access to Amazon Bedrock and the necessary permissions to create and manage knowledge bases.

* Step 1: Prepare Your Data
* Step 2: Create a Knowledge Base in Amazon Bedrock
* Step 3: Note the Knowledge Base ID

After the knowledge base is created, note down its Knowledge Base ID (kb_id), which will be used in our code.

Updating the Agent to Use RAG with CrewAI

We will modify our agent to include a custom tool that queries the Amazon Bedrock Knowledge Base. This allows the agent to retrieve up-to-date information during its reasoning process.

In [52]:
import boto3
# Initialize the Bedrock client
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name="us-east-1")
# Note: Replace "us-east-1" with your AWS region.

### FAIS Vector Store Set up:

In [53]:
import os
from uuid import uuid4
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")

documents = []
pdf_folder = '/docs/agents/open-source-l400/data/travel_guides'

# Loop through PDFs in the specified folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        file_path = os.path.join(pdf_folder, pdf_file)
        
        # Extract text from PDF
        reader = PdfReader(file_path)
        text_content = ""
        for page in reader.pages:
            text_content += page.extract_text() + "\n"
        
        # Create a Document instance
        doc = Document(
            page_content=text_content.strip(),
            metadata={}  # Leave metadata empty for now
        )
        documents.append(doc)

# Initialize FAISS vector store and embeddings
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
vector_store = FAISS.from_documents(documents, embeddings)

# Add unique IDs to documents and save the vector store
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['2c255607-bcb2-49db-8b8a-2e423799f684',
 '52ee05e7-1044-40cd-a415-5914d950409e',
 '98aec17d-e0a4-4a00-a246-d197cc9a5174',
 '04528d23-51b9-4210-91eb-c9a495e45f4d',
 '4cbc5228-ead4-410b-bbfb-22e7a85e115b',
 '7e23c104-57d1-4c13-a5b1-a0c277cc48f7',
 '4a90e0f8-e79f-4349-a099-df475ad71dc3',
 '097d5741-7e8f-400b-9db6-ab1299ae2584',
 '3e06e1b3-d1f0-4962-9957-a8d29d6986b0',
 'd6d4a793-754e-4ce2-a81c-aa173c0d048c',
 'fc70fe80-da26-4f75-86ee-96cf72eb35a6',
 '978e7202-292e-42ba-9e1f-2a1e760336c1',
 'bd643749-bf70-42cc-aee3-1c47e77815a0']

In [54]:
@tool("TravelExpertSearchEngine")
def query_knowledge_base(question: str) -> str:
    """Queries the Amazon Bedrock Knowledge Base for travel-related information."""
    try:
        res = vector_store.similarity_search(
        question,
        k=1,
        )        
        return res[0].page_content
    except KeyError:
        return "No data available"



### Update the Agent with the New Tool
We will update our agent to include the TravelExpert tool.

In [68]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0")

# Update the Agent
agent_with_rag = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations in the USA using only the travel guide available, first lookup cities using the tool to match user preferences and then use information from the search engine, nothing else.',
    backstory="""You are an experienced travel agent specializing in personalized travel recommendations. 
                 Your approach is as follows: 
                 Deduce which regions within the USA will have those activities listed by the user.
                 List major cities within that region
                 Only then use the tool provided to look up information, look up should be done by passing city highlights and activities.
                 Only suggest places that were extracted using the lookup tool,
              """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[query_knowledge_base],  # Include the RAG tool
    max_iter=3
)


### Update the task and set up the Crew

In [69]:
# Define the Task
task_with_rag = Task(
    description="Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: {preferences}",
    expected_output="A place where they can travel to along with recommendations on what to see and do while there.",
    agent=agent_with_rag
)


# Create the Crew
crew_with_rag = Crew(
    agents=[agent_with_rag],
    tasks=[task_with_rag],
    verbose=True,
)

2024-11-22 16:54:32,286 - 8615497536 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [70]:
# User input for travel preferences
user_input = {
    "preferences": "Where can I go for cowboy vibes, watch a rodeo, and a visit museums?"
}

# Execute the Crew
result_with_rag = crew_with_rag.kickoff(inputs=user_input)


# Agent: Travel Destination Researcher
## Task: Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: Where can I go for cowboy vibes, watch a rodeo, and a visit museums?


# Agent: Travel Destination Researcher
## Using tool: TravelExpertSearchEngine
## Tool Input: 
"{\"question\": \"cities in the US southwest for cowboy culture, rodeos, and museums\"}"
## Tool Output: 
Travel Guide: Dallas
Generated by Llama3.1 405B
 
Dallas, the vibrant heart of Texas, is a city that captivates with its dynamic blend of modern
sophistication and cowboy charm. As the ninth-largest city in the United States, Dallas dazzles visitors
with its towering skyscrapers, world-class museums, and thriving arts scene.
 
Explore the iconic Reunion Tower, where the observation deck offers panoramic views of the city
skyline. Immerse yourself in the rich history of the S

### Display the results

In [71]:
# Display the result
Markdown(result_with_rag.raw)

For those seeking a taste of authentic cowboy culture, rodeos, and fascinating museums, Dallas, Texas, is an excellent travel destination. Here are some key highlights and recommendations:

Cowboy Culture Attractions:
- Fort Worth Stockyards National Historic District: Experience the true cowboy spirit with daily cattle drives, cowboy museums, and authentic western shops and restaurants.
- Billy Bob's Texas: This massive honky-tonk is billed as the world's largest and offers live bull riding, country music concerts, and line dancing lessons.

Rodeos:
- Fort Worth Stock Show & Rodeo (January-February): One of the oldest and largest indoor rodeos, featuring bull riding, barrel racing, and other classic rodeo events.
- Mesquite Championship Rodeo (June-August): A popular summer rodeo series held every Friday and Saturday night, showcasing top professional cowboys and cowgirls.

Museums:
- Sixth Floor Museum at Dealey Plaza: Chronicling the life and legacy of President John F. Kennedy, with exhibits on his assassination in Dallas.
- Perot Museum of Nature and Science: An architectural marvel with interactive exhibits on dinosaurs, space exploration, and more.
- Dallas Museum of Art: Boasting an impressive collection of over 24,000 works spanning 5,000 years of human creativity.

Other Highlights:
- Visit the iconic Reunion Tower for panoramic city views and revolving restaurant.
- Explore the Dallas Arts District, featuring world-class venues like the Winspear Opera House and Nasher Sculpture Center.
- Sample delicious Tex-Mex cuisine and lively nightlife in the Deep Ellum neighborhood.

With its blend of cowboy heritage, exciting rodeo events, and rich cultural offerings, Dallas promises an unforgettable experience for travelers seeking a taste of the Wild West.